<a href="https://colab.research.google.com/github/apschlissel/w266-final-project/blob/main/Twitter_Topic_Category_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Sources:
# https://twarc-project.readthedocs.io/en/latest/twarc2_en_us/
# https://developer.twitter.com/en/docs/twitter-api/annotations/overview
# For command line stuff use ! before the command for it to work.

In [2]:
# Source: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=vz-jH8T_Uk2c
from google.colab import files
import json
import sys

In [3]:
pip install --upgrade twarc

     |████████████████████████████████| 56 kB 1.4 MB/s 
     |████████████████████████████████| 97 kB 2.4 MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34547 sha256=3c1ed1b4961d838dc1ac853803e64e5e93b44b543984c8bf8fc984f8de95df77
  Stored in directory: /root/.cache/pip/wheels/0d/c4/19/13d74440f2a571841db6b6e0a273694327498884dafb9cf978
Successfully built configobj
  Attempting uninstall: humanize
    Found existing installation: humanize 0.5.1
    Uninstalling humanize-0.5.1:
      Successfully uninstalled humanize-0.5.1


In [4]:
# Configuration Keys [KEEP]

# Steps:
# 1. Bearer Token: AAAAAAAAAAAAAAAAAAAAAB6MZQEAAAAA83pXM5U%2F%2Fxx3RMptSJU3pdBaw54%3DLWhdgT2cyplt2UkFk1H5wUvaIXwpCYZMAXTTCbjEMcdHbRskHO
# 2. no

# Optional:
# auth = tweepy.OAuthHandler("09bm0NBwQ66dep7yzsH5U13XP", "HWZ3DOoq7WZfOcY5JWmTImOXybMcVLPUkyOPfCEtVbHRray3UM" )
# auth.set_access_token("56585069-1vaful5MiHoyHT0SHvQVH0qDRZ5WwMv2h4muXe5Az", "ryy4e02nJDhxldrA0437FErbm1IsdF4wmetYpDD1Xmh14")
# !twarc2 configure

In [7]:
# Search Twitter for anything and it will output as a JSONL file (multiple lines of a JSON)
# Source: https://twittercommunity.com/t/how-to-save-a-json-file-using-twitter-api-v2-and-the-idle-of-python/157289/2
# Source: https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query

!twarc2 search --archive --start-time "2018-01-01" --end-time "2022-02-01" --limit 1000 "aye lang:en -is:reply -is:retweet" results.jsonl

# !twarc2 search --archive --limit 5000 "olympics (is:verified AND (-is:retweet OR -is:reply))" results.jsonl
# !twarc2 search --archive --start-time "2022-02-17" --end-time "2022-02-18" results.jsonl
# !twarc2 search --archive --start-time "2022-02-01" --end-time "2022-02-18" "from:tacobell" results.jsonl
# !twarc2 search --start-time 2022-02-17 --end-time 2022-02-18 tweets.jsonl

Set --limit of 1000 reached:   0% 15956167/128908800000 [00:21<48:46:52, 733959.76it/s]


In [8]:
# Open the jsonl file and put it into a list
with open('./results.jsonl', 'r') as json_file:
    json_list = list(json_file)

# I'm guessing each json line in jsonl contains 100 tweets
print("Length of json_list:", len(json_list))

Length of json_list: 11


In [9]:
# Write to CSV
# Source: https://www.pythontutorial.net/python-basics/python-write-csv-file/
import csv

# csv header
fieldnames = ['tweet_text', 'url', 'domain', 'entity', "one_hot_encoding"]

# Trying to write it into a list of dictionary elements:
rows = []

for l in range(len(json_list)):
  # Unwrap the list of dictionaries via json.loads()
  all_tweets_data = json.loads(json_list[l])["data"]
  # total_tweets_found.append(len(all_tweets_data))

  # print(all_tweets_data[0])
  # For each tweet in the result...
  for i in range(len(all_tweets_data)):
    one_tweet_dict = {}
    try:
      # Grab the context annotations dictionary only
      dict_of_context_annotations = all_tweets_data[i]["context_annotations"]
      one_tweet_dict["tweet_text"] = all_tweets_data[i]["text"]
      one_tweet_dict["url"] = "twitter.com/" + all_tweets_data[i]["author_id"] + "/status/" + all_tweets_data[i]["conversation_id"]
      domain_list = []
      entity_list = []
      # For each context annotation...
      for j in range(len(dict_of_context_annotations)):
        # Grab the domain id (label)
        if dict_of_context_annotations[j]["domain"]["name"] not in domain_list:
          domain_list.append(dict_of_context_annotations[j]["domain"]["name"])
        # # Grab the entity's name (label #2)
        if dict_of_context_annotations[j]["entity"]["name"] not in entity_list:
          entity_list.append(dict_of_context_annotations[j]["entity"]["name"])

      one_tweet_dict["domain"] = domain_list
      one_tweet_dict["entity"] = entity_list
      rows.append(one_tweet_dict)
      # print(one_tweet_dict)
    except:
      continue;

In [10]:
# Pretty print one item in the row.
print(json.dumps(rows[0], indent=4, sort_keys=False))

{
    "tweet_text": "Aye #Zendaya @Zendaya has truly ascended \n#TwitterGIFs https://t.co/aSG9MYDzvs",
    "url": "twitter.com/34296757/status/1488300920525971456",
    "domain": [
        "Person",
        "Musician",
        "Actor"
    ],
    "entity": [
        "Zendaya"
    ]
}


In [11]:
# The objective of this script is to fine tune BERT to be able to label a Domain into the following categories:
# TV Shows
# Music Genre
# Person
# Interests and Hobbies Vertical
# Brand
# Product

# Create a one hot encoding of the following 6 items:
search_list = ["TV Shows", "Music Genre", "Person", "Interests and Hobbies", "Brand", "Product"]
to_add_as_column = []

for a_dict in rows:
  # Source: https://stackoverflow.com/questions/62960181/how-to-append-a-list-of-zeros-onto-nonetype-list
  sublist = [0]*len(search_list)
  for domain in a_dict["domain"]:
    if domain in search_list:
      index = search_list.index(domain)
      sublist[index] = 1
  # to_add_as_column.append(sublist)
  a_dict["one_hot_encoding"] = sublist

In [12]:
# Pretty print one item in the row.
print(json.dumps(rows[0], indent=4, sort_keys=False))

{
    "tweet_text": "Aye #Zendaya @Zendaya has truly ascended \n#TwitterGIFs https://t.co/aSG9MYDzvs",
    "url": "twitter.com/34296757/status/1488300920525971456",
    "domain": [
        "Person",
        "Musician",
        "Actor"
    ],
    "entity": [
        "Zendaya"
    ],
    "one_hot_encoding": [
        0,
        0,
        1,
        0,
        0,
        0
    ]
}


In [13]:
with open('test.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

In [14]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 2.7 MB/s 


In [15]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 3.0 MB/s 
     |████████████████████████████████| 895 kB 43.2 MB/s 
     |████████████████████████████████| 596 kB 12.2 MB/s 
     |████████████████████████████████| 67 kB 3.5 MB/s 
     |████████████████████████████████| 6.5 MB 46.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [16]:
# Use BERT to classify tweets
# Source: https://colab.research.google.com/github/abhimishra91/transformers-tutorials/blob/master/transformers_multi_label_classification.ipynb#scrollTo=mZ7lTlkyaG7u

# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

The objective of this script is to fine tune BERT to be able to label a Domain  into the following categories:
- `TV Shows`
- `Music Genre`
- `Person`
- `Interests and Hobbies Vertical`
- `Brand`
- `Product`

In [17]:
df = pd.read_csv("./test.csv")
new_df = df[['tweet_text', 'one_hot_encoding']].copy()
new_df.head()

,tweet_text,one_hot_encoding
0,Aye #Zendaya @Zendaya has truly ascended \n#Tw...,"[0, 0, 1, 0, 0, 0]"
1,aye all i want for BHM is zero white ppl tik t...,"[0, 0, 0, 0, 0, 0]"
2,Aye shorty gotta be slanging D to have her out...,"[1, 0, 1, 1, 0, 0]"
3,Aye but do you have Wee Phillie getting an int...,"[0, 0, 1, 0, 0, 0]"
4,Aye! There's a friggen Easter Egg in my Yard!,"[0, 0, 0, 0, 0, 0]"


In [18]:
# Source: Keep on developing BERT from that^
# https://colab.research.google.com/github/abhimishra91/transformers-tutorials/blob/master/transformers_multi_label_classification.ipynb#scrollTo=mZ7lTlkyaG7u

In [19]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [24]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.tweet_text = dataframe.tweet_text
        self.targets = dataframe.one_hot_encoding
        self.max_len = max_len

    def __len__(self):
        return len(self.tweet_text)

    def __getitem__(self, index):
        tweet_text = str(self.tweet_text[index])
        tweet_text = " ".join(tweet_text.split())

        inputs = self.tokenizer.encode_plus(
            tweet_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [25]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=new_df.sample(frac=train_size,random_state=200)
test_dataset=new_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (332, 2)
TRAIN Dataset: (266, 2)
TEST Dataset: (66, 2)


In [26]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [28]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [29]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 6)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [30]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [31]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [41]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'][i].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [42]:
for epoch in range(EPOCHS):
    train(epoch)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


TypeError: ignored

In [ ]:
# ------------------------------------------------------------------------
# Source: https://stackoverflow.com/questions/50475635/loading-jsonl-file-as-json-objects
# Source: https://www.geeksforgeeks.org/python-convert-string-dictionary-to-dictionary/
# Source: https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query
# Source: https://stackabuse.com/writing-to-a-file-with-pythons-print-function/

with open('labeled_tweets_found2.txt', 'w') as f:

  labeled_tweet_total_count = 0
  # For every 100 tweets...
  for l in range(len(json_list)):
    # Unwrap the list of dictionaries via json.loads()
    all_tweets_data = json.loads(json_list[l])["data"]
    # total_tweets_found.append(len(all_tweets_data))

    # print(all_tweets_data[0])
    # For each tweet in the result...
    for i in range(len(all_tweets_data)):
      try:
        # Grab the context annotations dictionary only
        dict_of_context_annotations = all_tweets_data[i]["context_annotations"]
        # Increment labeled_tweet_total_count
        labeled_tweet_total_count += 1
        print("Tweet Text:", all_tweets_data[i]["text"], file=f)
        print("URL: twitter.com/" + all_tweets_data[i]["author_id"] + "/status/" + all_tweets_data[i]["conversation_id"], file=f)
        # For each context annotation...
        for j in range(len(dict_of_context_annotations)):
          # Grab the domain id (label)
          # print("Domain_ID:", dict_of_context_annotations[j]["domain"]["id"])
          print("Domain:", dict_of_context_annotations[j]["domain"]["name"], file=f)
          # # Grab the entity's name (label #2)
          # print("Entity_ID:", dict_of_context_annotations[j]["entity"]["id"])
          print("Entity:", dict_of_context_annotations[j]["entity"]["name"], file=f)
      except:
        continue;

  print("Total Tweets Found:", labeled_tweet_total_count, file=f)

In [ ]:
# Pretty print the json dictionary object
pretty_print = json.loads(json_list[0])
print(json.dumps(pretty_print, indent=4, sort_keys=True))

# for json_str in json_list[0]:
#     result = json.loads(json_str)
#     print(f"result: {result}")
#     print(isinstance(result, dict))

{
    "__twarc": {
        "retrieved_at": "2022-03-03T02:44:29+00:00",
        "url": "https://api.twitter.com/2/tweets/search/all?expansions=author_id%2Cin_reply_to_user_id%2Creferenced_tweets.id%2Creferenced_tweets.id.author_id%2Centities.mentions.username%2Cattachments.poll_ids%2Cattachments.media_keys%2Cgeo.place_id&tweet.fields=attachments%2Cauthor_id%2Ccontext_annotations%2Cconversation_id%2Ccreated_at%2Centities%2Cgeo%2Cid%2Cin_reply_to_user_id%2Clang%2Cpublic_metrics%2Ctext%2Cpossibly_sensitive%2Creferenced_tweets%2Creply_settings%2Csource%2Cwithheld&user.fields=created_at%2Cdescription%2Centities%2Cid%2Clocation%2Cname%2Cpinned_tweet_id%2Cprofile_image_url%2Cprotected%2Cpublic_metrics%2Curl%2Cusername%2Cverified%2Cwithheld&media.fields=alt_text%2Cduration_ms%2Cheight%2Cmedia_key%2Cpreview_image_url%2Ctype%2Curl%2Cwidth%2Cpublic_metrics&poll.fields=duration_minutes%2Cend_datetime%2Cid%2Coptions%2Cvoting_status&place.fields=contained_within%2Ccountry%2Ccountry_code%2Cfull_name%